# Code Generation and Evaluation

In [30]:
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from datasets import load_dataset
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full, )
from helpers import get_tokenizer, get_model, generate_response, clean_code, write_jsonl

In [32]:
# select model, propmt, temperature, top_p
models = [
    ('open-mistral-7b', 'mistral_7b'),
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    ('open-codestral-mamba', 'codestral_mamba'),
    ]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx       = 4
prompt_idx      = 0
temperature_idx = 0
top_p_idx       = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:     {model_name}')
print(f'Model nickname: {model_nickname}')
print(f'Prompt name:    {my_prompt_label}')
print(f'Prompt:\n{my_prompt.strip().format("Starter Code")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:     open-codestral-mamba
Model nickname: codestral_mamba
Prompt name:    complete_code_prompt_basic
Prompt:
Complete the following Python code:
Starter Code

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [33]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-03-30 05:09:39,797  INFO  *****     JOB CONFIGURATION     *****
2025-03-30 05:09:39,797  INFO  MODEL: open-codestral-mamba
2025-03-30 05:09:39,798  INFO  MODEL NICKNAME: codestral_mamba
2025-03-30 05:09:39,798  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-03-30 05:09:39,799  INFO  GENERATED CODE SAVED IN: logs_local/codestral_mamba_complete_code_prompt_basic_temperature1.0_topP1.0_completions_20250330_050939_7966.jsonl
2025-03-30 05:09:39,799  INFO  THIS LOG FILE: logs_local/logs/codestral_mamba_complete_code_prompt_basic_temperature1.0_topP1.0_log_20250330_050939_7966.log
2025-03-30 05:09:39,799  INFO  PROMPT:
Complete the following Python code:
Starter Code

2025-03-30 05:09:39,800  INFO  *****     END OF JOB CONFIGURATION     *****


In [34]:
dataset = load_dataset("openai/openai_humaneval")
counter = 0
for item in dataset["test"]:
    if counter == 1: break
    counter += 1
    for k,v in item.items():
        print(f'{k}:\n{v}\n')
    print('\n', '='*75, '\n', sep='')

task_id:
HumanEval/0

prompt:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """


canonical_solution:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False


test:


METADATA = {
    'author': 'jt',
    'dataset': 'test'
}


def check(candidate):
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True
    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.95) == True
    assert candidate([1.0, 2.0, 5.9, 4.0, 5.0], 0.8) == False

In [35]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [36]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [37]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-03-30 05:09:42,231  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It serves as the state capital and the seat of the California State Legislature.


In [38]:
num_samples_per_task = 1
completions  = []

for item in dataset["test"]:
    logging.info(item['task_id'])
    #if item['task_id'] == 'HumanEval/5':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = my_prompt.lstrip().format( item['prompt'] )
        completion = generate_response_with_backoff( model_name, full_prompt )
        completions.append( {'task_id': item['task_id'], 'completion': completion} )
        write_jsonl(results_file, completions)
        
        logging.info('PROMPT:\n' + full_prompt + '\n')
        logging.info('COMPLETION:\n' + completion + '\n')
        logging.info(f"Time elapsed: {(time.time() - start_time):.4f} seconds\n")

2025-03-30 05:09:42,298  INFO  HumanEval/0
2025-03-30 05:09:45,301  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:09:45,313  INFO  PROMPT:
Complete the following Python code:
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



2025-03-30 05:09:45,314  INFO  COMPLETION:
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers)):
        for j in range(i+1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False

print(has_close_elements([1.0, 2.0, 3.0], 0.5))
print(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2

2025-03-30 05:10:04,571  INFO  COMPLETION:
def parse_nested_parens(paren_string: str) -> List[int]:
    def get_max_depth(paren_string: str) -> int:
        max_depth = 0
        depth = 0
        for char in paren_string:
            if char == '(':
                depth += 1
                max_depth = max(max_depth, depth)
            elif char == ')':
                depth -= 1
        return max_depth

    return [get_max_depth(group) for group in paren_string.split()]

print(parse_nested_parens('(()()) ((())) () ((())()())'))

2025-03-30 05:10:04,571  INFO  Time elapsed: 1.9238 seconds

2025-03-30 05:10:04,573  INFO  HumanEval/7
2025-03-30 05:10:06,843  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:10:06,857  INFO  PROMPT:
Complete the following Python code:
from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain give

2025-03-30 05:10:25,046  INFO  Time elapsed: 1.9237 seconds

2025-03-30 05:10:25,047  INFO  HumanEval/14
2025-03-30 05:10:27,182  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:10:27,193  INFO  PROMPT:
Complete the following Python code:
from typing import List


def all_prefixes(string: str) -> List[str]:
    """ Return list of all prefixes from shortest to longest of the input string
    >>> all_prefixes('abc')
    ['a', 'ab', 'abc']
    """



2025-03-30 05:10:27,194  INFO  COMPLETION:
def all_prefixes(string: str) -> List[str]:
    return [string[:i] for i in range(1, len(string) + 1)]

# Test the function
print(all_prefixes('abc'))  # Output: ['a', 'ab', 'abc']

2025-03-30 05:10:27,196  INFO  Time elapsed: 1.0461 seconds

2025-03-30 05:10:27,197  INFO  HumanEval/15
2025-03-30 05:10:30,255  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:10:30,269  INFO  PROMPT:
Complete the fol

2025-03-30 05:10:47,061  INFO  Time elapsed: 2.3710 seconds

2025-03-30 05:10:47,063  INFO  HumanEval/21
2025-03-30 05:10:50,534  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:10:50,547  INFO  PROMPT:
Complete the following Python code:
from typing import List


def rescale_to_unit(numbers: List[float]) -> List[float]:
    """ Given list of numbers (of at least two elements), apply a linear transform to that list,
    such that the smallest number will become 0 and the largest will become 1
    >>> rescale_to_unit([1.0, 2.0, 3.0, 4.0, 5.0])
    [0.0, 0.25, 0.5, 0.75, 1.0]
    """



2025-03-30 05:10:50,549  INFO  COMPLETION:
Sure, here is a Python code that solves your problem:

```python
from typing import List

def rescale_to_unit(numbers: List[float]) -> List[float]:
    min_val = min(numbers)
    max_val = max(numbers)

    if min_val == max_val:
        return [0.5] * len(numbers)

    return [(num - min_val) / (max_val - min_v

2025-03-30 05:11:10,103  INFO  COMPLETION:
def concatenate(strings: List[str]) -> str:
    """ Concatenate list of strings into a single string
    >>> concatenate([])
    ''
    >>> concatenate(['a', 'b', 'c'])
    'abc'
    """
    return ''.join(strings)

2025-03-30 05:11:10,104  INFO  Time elapsed: 1.0451 seconds

2025-03-30 05:11:10,105  INFO  HumanEval/29
2025-03-30 05:11:12,320  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:11:12,334  INFO  PROMPT:
Complete the following Python code:
from typing import List


def filter_by_prefix(strings: List[str], prefix: str) -> List[str]:
    """ Filter an input list of strings only for ones that start with a given prefix.
    >>> filter_by_prefix([], 'a')
    []
    >>> filter_by_prefix(['abc', 'bcd', 'cde', 'array'], 'a')
    ['abc', 'array']
    """



2025-03-30 05:11:12,335  INFO  COMPLETION:
from typing import List

def filter_by_prefix(strings: List[str], prefix: str) -> List[str]:

...

2025-03-30 05:17:37,429  INFO  HumanEval/139
2025-03-30 05:17:39,624  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:17:39,639  INFO  PROMPT:
Complete the following Python code:

def special_factorial(n):
    """The Brazilian factorial is defined as:
    brazilian_factorial(n) = n! * (n-1)! * (n-2)! * ... * 1!
    where n > 0

    For example:
    >>> special_factorial(4)
    288

    The function will receive an integer as input and should return the special
    factorial of this integer.
    """



2025-03-30 05:17:39,640  INFO  COMPLETION:
def special_factorial(n):
    if n == 0:
        return 1
    else:
        result = 1
        for i in range(n, 0, -1):
            result *= special_factorial(i)
        return result

print(special_factorial(4))  # Output: 288

2025-03-30 05:17:39,641  INFO  Time elapsed: 1.1061 seconds

2025-03-30 05:17:39,642  INFO  HumanEval/140
2025-03-30 05:17:43,524  INFO  HTTP Request: POST https://api

2025-03-30 05:17:56,922  INFO  Time elapsed: 3.1521 seconds

2025-03-30 05:17:56,923  INFO  HumanEval/144
2025-03-30 05:18:02,150  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:18:02,173  INFO  PROMPT:
Complete the following Python code:

def simplify(x, n):
    """Your task is to implement a function that will simplify the expression
    x * n. The function returns True if x * n evaluates to a whole number and False
    otherwise. Both x and n, are string representation of a fraction, and have the following format,
    <numerator>/<denominator> where both numerator and denominator are positive whole numbers.

    You can assume that x, and n are valid fractions, and do not have zero as denominator.

    simplify("1/5", "5/1") = True
    simplify("1/6", "2/1") = False
    simplify("7/10", "10/2") = False
    """



2025-03-30 05:18:02,174  INFO  COMPLETION:
To solve this problem, we need to convert the string representation of the f

2025-03-30 05:18:20,273  INFO  COMPLETION:
def bf(planet1, planet2):
    solar_system = {
        "Mercury": 0, "Venus": 1, "Earth": 2, "Mars": 3, "Jupiter": 4,
        "Saturn": 5, "Uranus": 6, "Neptune": 7
    }

    try:
        start = solar_system[planet1]
        end = solar_system[planet2]
    except KeyError:
        return ()

    if start > end:
        start, end = end, start

    return tuple(planet for planet, distance in solar_system.items() if start < distance < end)

print(bf("Jupiter", "Neptune"))
print(bf("Earth", "Mercury"))
print(bf("Mercury", "Uranus"))
print(bf("Earth", "Pluto"))

2025-03-30 05:18:20,274  INFO  Time elapsed: 2.6894 seconds

2025-03-30 05:18:20,276  INFO  HumanEval/149
2025-03-30 05:18:23,143  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:18:23,158  INFO  PROMPT:
Complete the following Python code:

def sorted_list_sum(lst):
    """Write a function that accepts a list of strings as a parameter,


2025-03-30 05:18:37,903  INFO  COMPLETION:
Here is the Python code to solve this problem:

```python
def cycpattern_check(a, b):
    for i in range(len(b)):
        temp = b[i:] + b[:i]
        if temp in a:
            return True
    return False
```
This function works by generating all possible rotations of the string `b`, and then checks if any of these rotations are substrings of `a`. If it finds a match, it returns `True`; otherwise, it returns `False`.

2025-03-30 05:18:37,904  INFO  Time elapsed: 1.5495 seconds

2025-03-30 05:18:37,905  INFO  HumanEval/155
2025-03-30 05:18:40,752  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:18:40,766  INFO  PROMPT:
Complete the following Python code:

def even_odd_count(num):
    """Given an integer. return a tuple that has the number of even and odd digits respectively.

     Example:
        even_odd_count(-12) ==> (1, 1)
        even_odd_count(123) ==> (1, 2)
    """



2025-03-30 05:1

2025-03-30 05:18:57,981  INFO  COMPLETION:
def do_algebra(operator, operand):
    expression = str(operand[0])
    for i in range(len(operator)):
        expression += ' ' + operator[i] + ' ' + str(operand[i+1])
    return eval(expression)

operator = ['+', '*', '-']
operand = [2, 3, 4, 5]
print(do_algebra(operator, operand)) # Output: 9

2025-03-30 05:18:57,981  INFO  Time elapsed: 1.3934 seconds

2025-03-30 05:18:57,983  INFO  HumanEval/161
2025-03-30 05:19:00,520  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-30 05:19:00,534  INFO  PROMPT:
Complete the following Python code:

def solve(s):
    """You are given a string s.
    if s[i] is a letter, reverse its case from lower to upper or vise versa, 
    otherwise keep it as it is.
    If the string contains no letters, reverse the string.
    The function should return the resulted string.
    Examples
    solve("1234") = "4321"
    solve("ab") = "AB"
    solve("#a@C") = "#A@c"
    """


In [39]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)